In [1]:
!pip install streamlit

In [2]:


%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor

# -----------------------------
# Load Data
# -----------------------------
@st.cache_data
def load_data():
    df = pd.read_csv("combined_output.csv")
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
    df['PM2.5'].fillna(df['PM2.5'].median(), inplace=True)
    features = ['TEMP', 'PRES', 'DEWP', 'WSPM', 'SO2', 'NO2', 'CO', 'O3']
    df[features] = df[features].fillna(df[features].median())
    df = df[df['PM2.5'] < 500]
    return df

df = load_data()




# -----------------------------
# Train the model
# -----------------------------
features = ['TEMP', 'PRES', 'DEWP', 'WSPM', 'SO2', 'NO2', 'CO', 'O3']
X = df[features]
y = df['PM2.5']
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X, y)

# -----------------------------
# App Navigation & Layout
# -----------------------------
st.sidebar.title("📌 Navigation")
section = st.sidebar.radio("Go to Section", ["📋 Data Overview", "🌍 Dashboard", "🔬 EDA"])

st.title("🌍 PM2.5 Predictor & Air Quality Visual Dashboard")
st.markdown("**📍 Use sliders to predict PM2.5 and explore trends with charts.**")

# -----------------------------
# 📋 Data Overview Section
# -----------------------------
if section == "📋 Data Overview":
    st.header("📋 Data Overview")

    st.subheader("🔍 Dataset Preview")
    st.dataframe(df.head())

    st.markdown(f"**🧮 Shape of dataset:** `{df.shape[0]} rows × {df.shape[1]} columns`")

    st.subheader("🔢 Data Types and Missing Values")
    dtypes_df = pd.DataFrame({
        "Data Type": df.dtypes,
        "Missing Values": df.isnull().sum(),
        "Missing (%)": (df.isnull().sum() / len(df) * 100).round(2)
    })
    st.dataframe(dtypes_df)

    st.subheader("📊 Statistical Summary")
    st.dataframe(df.describe().T)

# -----------------------------
# 🔮 Dashboard Section
# -----------------------------
elif section == "🔮 Dashboard":

    # User Input
    st.sidebar.header("🧪 Input Values")

    temp = st.sidebar.slider("Temperature (°C)", -8.0, 50.0, 20.0)
    pres = st.sidebar.slider("Pressure (hPa)", 980.0, 1040.0, 1010.0)
    dewp = st.sidebar.slider("Dew Point", -40.0, 30.0, 5.0)
    wspm = st.sidebar.slider("Wind Speed", 0.0, 10.0, 2.0)
    so2 = st.sidebar.slider("SO₂", 0.0, 100.0, 10.0)
    no2 = st.sidebar.slider("NO₂", 0.0, 150.0, 30.0)
    co = st.sidebar.slider("CO", 0.0, 5000.0, 900.0)
    o3 = st.sidebar.slider("O₃", 0.0, 300.0, 50.0)

    input_data = np.array([[temp, pres, dewp, wspm, so2, no2, co, o3]])
    predicted_pm25 = model.predict(input_data)[0]

    # Prediction Output
    st.subheader("🔮 Predicted PM2.5 Level:")
    st.success(f"{predicted_pm25:.2f} µg/m³")

    # Data Visualisations
    st.header("📈 Data Visualisations")
    df.set_index("datetime", inplace=True)
    monthly_pm = df['PM2.5'].resample('M').mean()

    st.markdown("### 📅 Monthly PM2.5 Trend")
    st.line_chart(monthly_pm)

    st.markdown("### 📊 PM2.5 Distribution Histogram")
    fig1, ax1 = plt.subplots()
    sns.histplot(df['PM2.5'], bins=50, kde=True, ax=ax1, color='skyblue')
    plt.xlabel("PM2.5 (µg/m³)")
    plt.ylabel("Frequency")
    plt.title("Distribution of PM2.5")
    st.pyplot(fig1)

    st.markdown("### 🔥 Correlation Heatmap")
    fig2, ax2 = plt.subplots(figsize=(10, 8))
    sns.heatmap(df[features + ['PM2.5']].corr(), annot=True, fmt=".2f", cmap='coolwarm', ax=ax2)
    plt.title("Correlation Matrix")
    st.pyplot(fig2)

# -----------------------------
# 🔬 EDA Section
# -----------------------------
elif section == "🔬 EDA":
    st.title("🔬 Exploratory Data Analysis (EDA)")
    sns.set_theme(style="whitegrid")

    st.header(" PM2.5 Distribution")
    fig1, ax1 = plt.subplots(figsize=(8, 4))
    sns.histplot(df["PM2.5"].dropna(), bins=40, kde=True, color="blue", ax=ax1)
    ax1.set_title("Distribution of PM2.5 (µg/m³)", fontsize=14)
    ax1.set_xlabel("PM2.5", fontsize=12)
    st.pyplot(fig1)

    st.header(" Monthly Trend of PM2.5")
    df["date"] = pd.to_datetime(df.index, errors='coerce')
    df = df.set_index("date")
    monthly_avg = df["PM2.5"].resample("M").mean()
    fig3, ax3 = plt.subplots(figsize=(10, 4))
    monthly_avg.plot(ax=ax3, color="blue", marker="o", linestyle="--")
    ax3.set_title("Monthly Average PM2.5 (µg/m³)", fontsize=14)
    ax3.set_ylabel("PM2.5 (µg/m³)")
    st.pyplot(fig3)

    st.header(" Dynamic Pollutant Trends")
    selected_cols = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'WSPM']
    pollutant = st.selectbox("Select a pollutant to visualize its trend", selected_cols[1:])
    if pollutant:
        fig4, ax4 = plt.subplots(figsize=(10, 4))
        df[pollutant].resample("M").mean().plot(ax=ax4, color="crimson", linestyle="-", marker=".")
        ax4.set_title(f"Monthly Average {pollutant}", fontsize=14)
        ax4.set_ylabel(f"{pollutant} (µg/m³ or mg/m³)")
        st.pyplot(fig4)

    st.header(" Outlier Detection (Boxplot)")
    box_cols = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3']
    selected_box = st.selectbox("Select a feature for boxplot", box_cols)
    if selected_box:
        fig5, ax5 = plt.subplots()
        sns.boxplot(y=df[selected_box].dropna(), color="red", ax=ax5)
        ax5.set_title(f"Boxplot of {selected_box}", fontsize=14)
        st.pyplot(fig5)


Overwriting app.py


In [3]:
!wget -q -O - ipv4.icanhazip.com

34.168.254.156


In [ ]:

!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸⠼⠴⠦⠧
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.168.254.156:8501

⠇your url is: https://silly-rivers-sniff.loca.lt
/content/app.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['PM2.5'].fillna(df['PM2.5'].median(), inplace=True)
2025-05-15 13:53:59.232 Serialization of dataframe to Arrow table was unsuccessful. Applying automatic fixes for column types to make the dataframe Arrow-compatible.
Traceback (m